5章1節,2節を実行する

# レシピ5.1 Juliaのサブタイプを理解する

Juliaのサブタイプの機能と、サブタイプを適切に扱うためのメソッドシグネチャの書き方について説明する

ガウス整数(実部と虚部が整数の複素数)にメタデータを追加した型を定義する。

Point型を定義して、配列に格納する

パラメータT, Sを持つPoint型を作る

In [1]:
# T<:Integerは、Tの型がIntegerに含まれることを意味する
struct Point{T<:Integer, S<:AbstractString}
    pos::Complex{T}
    label::S
end

型を定義しただけでは1つしかコンストラクタが出来ないので、2つのカスタムコンストラクタを追加する。  
2つの引数x,yをとり、それらを整数型に変換し、変換出来たら共通の方にpromoteする。  
次に呼び出すもう一つのコンストラクタの第一引数と第二引数が同じ方でなければならないから。

In [2]:
Point(x::T, y::T, label::S) where {T<:Integer, S<:AbstractString} = Point{T,S}(Complex(x,y), label)

Point

インスタンスの作成

In [3]:
# SubString: 部分列。Juliaは1文字をchar型と認識するので、String型として認識させている
# Substring(string, start, end)
p1 = Point(1, 0, "1")
p2 = Point(1, 0, SubString("1", 1))
p3 = Point(true, false, "1")
p4 = Point(2, 0, "2")

Point{Int64,String}(2 + 0im, "2")

In [4]:
p1.pos

1 + 0im

In [5]:
p3.label

"1"

配列の作成

In [6]:
[p1, p2, p3, p4]

4-element Array{Point,1}:
 Point{Int64,String}(1 + 0im, "1")
 Point{Int64,SubString{String}}(1 + 0im, "1")
 Point{Bool,String}(Complex(true,false), "1")
 Point{Int64,String}(2 + 0im, "2")

In [7]:
[p1, p2]

2-element Array{Point{Int64,S} where S<:AbstractString,1}:
 Point{Int64,String}(1 + 0im, "1")
 Point{Int64,SubString{String}}(1 + 0im, "1")

In [8]:
[p1, p3]

2-element Array{Point{T,String} where T<:Integer,1}:
 Point{Int64,String}(1 + 0im, "1")
 Point{Bool,String}(Complex(true,false), "1")

In [9]:
[p1, p4]

2-element Array{Point{Int64,String},1}:
 Point{Int64,String}(1 + 0im, "1")
 Point{Int64,String}(2 + 0im, "2")

このPoint型の配列を受け取り、配列中のPointの合計に空文字列のラベルを付けたインスタンスを返すメソッドを作成

In [10]:
sumpoint1(v::AbstractVector{Point}) = Point(sum(p.pos for p in v), "")

sumpoint1 (generic function with 1 method)

In [11]:
# 関数の内容をそのまま書くと通る
Point(sum(p1.pos for p in [p1,p2]), "")

Point{Int64,String}(2 + 0im, "")

In [12]:
# 関数を定義して実行するとエラーが出る, Point型そのものではないため
sumpoint1([p1,p2])

MethodError: MethodError: no method matching sumpoint1(::Array{Point{Int64,S} where S<:AbstractString,1})
Closest candidates are:
  sumpoint1(!Matched::AbstractArray{Point,1}) at In[10]:1

In [13]:
sumpoint2(v::AbstractVector{<:Point}) = Point(sum(p.pos for p in v), "")

sumpoint2 (generic function with 1 method)

In [14]:
sumpoint2([p1,p2])

Point{Int64,String}(2 + 0im, "")

別の関数を定義する

In [15]:
# 入力する型を変えて同じ関数名を定義すると、関数の挙動を追加出来る
foo(p::Point) = "一般的な定義"
foo(p::Point{Int, <:AbstractString}) = "Intが渡された際のデフォルト"
foo(p::Point{<:Integer, String}) = "Stringが渡された際のデフォルト"

foo (generic function with 3 methods)

色々なインスタンスで試す

In [16]:
foo(Point(true, true, s"12"))

"一般的な定義"

In [17]:
# s"hogehoge"は、正規表現で置換する対象の文字列を作成する. Pythonでいうところのr"hogehoge"
foo(Point(1, 1, s"12"))

"Intが渡された際のデフォルト"

In [18]:
foo(Point(true, true, "12"))

"Stringが渡された際のデフォルト"

In [19]:
foo(Point(1, 1, "12"))

MethodError: MethodError: foo(::Point{Int64,String}) is ambiguous. Candidates:
  foo(p::Point{Int64,#s1} where #s1<:AbstractString) in Main at In[15]:3
  foo(p::Point{#s1,String} where #s1<:Integer) in Main at In[15]:4
Possible fix, define
  foo(::Point{Int64,String})

型の組み合わせに対してそれぞれ別にメソッドを定義する必要があることがわかる。

In [20]:
foo(p::Point{Int, String}) = "厳密に型を指定したメソッド"

foo (generic function with 4 methods)

In [21]:
foo(Point(1, 1, "12"))

"厳密に型を指定したメソッド"

通状のコンストラクタは、渡されたデータを保持出来る範囲で最も狭いデータ型を見つけようとする。  
コレクションに異なるデータ型が回ることがありうるなら、コレクションを作る時点で適切な型を指定しておく必要がある。

In [22]:
# push!とは: https://goropikari.hatenablog.com/entry/julia_array_implement
# pushとは、配列に要素を追加する関数
# この場合、p1の型、Point{Int64,String}に対してp2の型Point{Int64,SubString{String}}を混在させてしまう
push!([p1], p2)

MethodError: MethodError: Cannot `convert` an object of type 
  Point{Int64{},SubString{String}} to an object of type 
  Point{Int64{},String}
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T at essentials.jl:171
  Point{Int64,String}(::Any, !Matched::Any) where {T<:Integer, S<:AbstractString} at In[1]:3

In [23]:
# このArrayの型はArray{Point{Int64,String},1}, こちらの方が型が厳密になっている
[p1]

1-element Array{Point{Int64,String},1}:
 Point{Int64,String}(1 + 0im, "1")

In [24]:
# このArrayの型はArray{Point,1}
Point[p1]

1-element Array{Point,1}:
 Point{Int64,String}(1 + 0im, "1")

In [25]:
push!(Point[p1], p2)

2-element Array{Point,1}:
 Point{Int64,String}(1 + 0im, "1")
 Point{Int64,SubString{String}}(1 + 0im, "1")

Juliaではほとんどのパラメータ型が非変。  
型パラメータをサブタイプにしても、その型パラメータでパラメータ化された型の方はサブタイプにならない。

In [26]:
# Int型はInteger型に内包される
Int <: Integer

true

In [27]:
# これは無理
Point{Int, String} <: Point{Integer, String}

false

In [28]:
# <: 演算子を使って、型のサブタイプを許容する
Point{Int, String} <: Point{<:Integer, String}

true

In [29]:
# whereを使った等価な書き方
Point{Int, String} <: Point{T, String} where T<: Integer

true

2つ以上の型パラメータでパラメータ化された型は、型パラメータ毎にサブタイプに制約できる。

In [30]:
# 最初の引数をIntegerからIntに制約
Point{Int}

Point{Int64,S} where S<:AbstractString

In [31]:
# Signed: 符号付き整数
# https://docs.julialang.org/en/v1/base/numbers/#Core.Signed
Point{<:Signed, String}

Point{#s1,String} where #s1<:Signed

In [32]:
Point{Int}{String}

Point{Int64,String}

In [33]:
Point{Int, String}

Point{Int64,String}

In [34]:
# 再掲、これはエラーになる. p1によって型がPoint{Int64,String}に制約されてしまうため.
sumpoint1([p1,p2])

MethodError: MethodError: no method matching sumpoint1(::Array{Point{Int64,S} where S<:AbstractString,1})
Closest candidates are:
  sumpoint1(!Matched::AbstractArray{Point,1}) at In[10]:1

In [35]:
# 先に、型を2-element Array{Point,1} とすればOK
# p1とp2を保持する配列の型を明示的に指定している
sumpoint1(Point[p1,p2])

Point{Int64,String}(2 + 0im, "")

関数に指定されているメソッドをチェックするにはmethods関数を用いる

In [36]:
methods(foo)

# 4 methods for generic function "foo":
[1] foo(p::Point{Int64,String}) in Main at In[20]:1
[2] foo(p::Point{Int64,#s1} where #s1<:AbstractString) in Main at In[15]:3
[3] foo(p::Point{#s1,String} where #s1<:Integer) in Main at In[15]:4
[4] foo(p::Point) in Main at In[15]:2

Juliaの型は不変だが、タプルは共変

In [37]:
Tuple{Point{Int, String}, Point{Bool, SubString{String}}} <: Tuple{Point{Int}, Point}

true

In [38]:
# Vararg: 任意の数の末尾の要素を表す特殊な型
# Vararg{T,N}型は，T型の正確にN個の要素に対応します．Vararg{T}型は、T型の0個以上の要素に対応します。
sumpoint_tuple(v::Tuple{Vararg{Point}}) = Point(sum(p.pos for p in v), "")

sumpoint_tuple (generic function with 1 method)

In [39]:
(p1,p2,p3)

(Point{Int64,String}(1 + 0im, "1"), Point{Int64,SubString{String}}(1 + 0im, "1"), Point{Bool,String}(Complex(true,false), "1"))

In [40]:
sumpoint_tuple((p1,p2,p3))

Point{Int64,String}(3 + 0im, "")

# レシピ5.2 多重ディスパッチで動作を切り替える

多重ディスパッチ: 関数に入力された引数に応じて、対応するメソッドを切り替えること

いくつかの異なるコンテント型を持つDataFrameを効率的に扱う方法を紹介する

In [41]:
using DataFrames

In [42]:
df = DataFrame(
    s = categorical(["a", "b", "c"]),
    n = 1.0:3.0,
    f = [sin, cos, missing]
)

,s,n,f
,Cat…,Float64,Function?
1,a,1.0,sin (generic function with 12 methods)
2,b,2.0,cos (generic function with 12 methods)
3,c,3.0,missing


In [43]:
simpledescribe(v) = "unknown type"
simpledescribe(v::Vector{<:Number}) = "numeric"
simpledescribe(v::CategoricalArray) = "categorical"

simpledescribe (generic function with 3 methods)

データフレームを読み取って表示する関数を定義する

In [44]:
simpledisplay(df) =
    foreach(x -> println(x[1], ": ", simpledescribe(x[2])),
    collect(pairs(eachcol(df)))
)

simpledisplay (generic function with 1 method)

In [45]:
simpledisplay(df)

s: categorical
n: numeric
f: unknown type


関数: 引数の組を返り値にマップするオブジェクト  
引数として渡された値の型に応じて複数の挙動を定義することが出来る  
関数に対して定義された、それぞれの挙動をメソッドと呼ぶ  

In [46]:
methods(simpledescribe)

# 3 methods for generic function "simpledescribe":
[1] simpledescribe(v::Array{#s3,1} where #s3<:Number) in Main at In[43]:2
[2] simpledescribe(v::CategoricalArray) in Main at In[43]:3
[3] simpledescribe(v) in Main at In[43]:1

In [47]:
# ;をつけると改行される. dfを出力しなくなる
df = DataFrame(x=1:10^6);

In [48]:
# eltype
# 与えられた型のコレクションを反復処理することで生成される要素の型を決定します。
# 辞書型の場合、これはPair{KeyType,ValType}になります。
# eltype(x) = eltype(typeof(x))という定義は、型の代わりにインスタンスを渡すことができるように便宜的に提供されています。
# ただし、新しい型の場合は型の引数を受け付ける形式を定義する必要があります。
function helper(x)
    s = zero(eltype(x))
    for v in x
        s += v
    end
    s
end

helper (generic function with 1 method)

In [49]:
# 入力されるdfの型は特に指定されていない
# df.col は df[!, col] のように動作します。
function fun1(df)
    s = zero(eltype(df[!,1]))
    for v in df[!,1]
        s += v
    end
    s
end

fun1 (generic function with 1 method)

In [50]:
# 予めdfの型を指定しておく
fun2(df) = helper(df[!,1])

fun2 (generic function with 1 method)

In [51]:
using BenchmarkTools

In [52]:
@btime fun1(df)

  71.382 ms (3998948 allocations: 76.28 MiB)


500000500000

In [53]:
@btime fun2(df)

  270.913 μs (1 allocation: 16 bytes)


500000500000

fun2は予めxの型がわかっている状態でコンパイルされるヘルパ関数で実際の仕事をしているため、  
Juliaがその方に対して効率的に動作するコードを生成できている